### 有新旧对比的

In [ ]:
import pandas as pd
import numpy as np
import os
import csv

class CheckSize():
    
    def __init__(self, newfolder, oldfolder, case, year, title):
        self.newfolder = newfolder
        self.oldfolder = oldfolder
        self.case = case
        self.year = year
        self.title = title
        
    def checkrows(self):  #每次调取这个函数是一年的
        
        new_path = self.newfolder  #包含年份的一整条绝对路径
        old_path = self.oldfolder
        NewDatePath = []  #筛选过的完整路径集合
        OldDatePath = []

        old_datelist = os.listdir(old_path)   #仅遍历获取该年份下所有的日期文件夹名字的集合
        # orderflow五分钟线
        if self.case == 1:
            for date_i in old_datelist:  #筛出两边都有的时间文件
                if os.path.exists(new_path + '/' + date_i) == True:
                    NewDatePath.append(new_path + '/' + date_i) 
                    OldDatePath.append(old_path + '/' + date_i)
        # orderflow日线
        if self.case == 2:
            for date_i in old_datelist: 
                if os.path.exists(new_path + '/' + date_i + '/' + date_i + '.txt') == True:
                    NewDatePath.append(new_path + '/' + date_i + '/' + date_i + '.txt') 
                    OldDatePath.append(old_path + '/' + date_i + '/daily')
    
        headers = ['date', 'time', 'new_rows', 'old_rows']
        OneYear_log = open("/home/yuqin/work/2020Compare/0114/%s%sCheckRows.csv" %(self.year,self.title), "a")
        log = csv.DictWriter(OneYear_log, headers)
        for n in range(len(NewDatePath)):
            
            if self.case == 1:
                NewTimePath = []
                OldTimePath = []
                old_timelist = os.listdir(OldDatePath[n]) 
                for time_i in old_timelist:
                    if os.path.exists(NewDatePath[n] + '/' + time_i) == True:
                        NewTimePath.append(NewDatePath[n] + '/' + time_i) 
                        OldTimePath.append(OldDatePath[n] + '/' + time_i)
                    
                for m in range(len(OldTimePath)):
                    getpath_new = NewTimePath[m]
                    getpath_old = OldTimePath[m]
                    print(getpath_new)
                    print(getpath_old)
                    new = pd.read_csv(getpath_new, sep = ',')
                    old = pd.read_csv(getpath_old, sep = ',')
                    new.set_index('SecuCode', inplace=True)
                    old.set_index('SecuCode', inplace=True)
                    new_rows = len(new.index)
                    old_rows = len(old.index)
                    if (new_rows != old_rows) or (new_rows < 500) or (old_rows < 500):
                        d = getpath_old
                        time = d[ d.rindex( '/' ) + 1 : len( d ) ] 
                        withoutLastStr = d[ 0 : d.rindex( '/' ) ]
                        date= withoutLastStr[ withoutLastStr.rindex( '/' ) + 1 : len( withoutLastStr ) ]
                        log.writerow({'date':str(date), 'time':str(time), 'new_rows':str(new_rows), 'old_rows':str(old_rows)})           
                    
            if self.case == 2:
                getpath_new = NewDatePath[n]
                getpath_old = OldDatePath[n]
                print(getpath_new)
                print(getpath_old)
                new = pd.read_csv(getpath_new, sep = ',')
                old = pd.read_csv(getpath_old, sep = ',')
                new.set_index('SecuCode', inplace=True)
                old.set_index('SecuCode', inplace=True)
                new_rows = len(new.index)
                old_rows = len(old.index)
                if (new_rows != old_rows) or (new_rows < 500) or (old_rows < 500):
                    d = getpath_new
                    withoutLastStr = d[ 0 : d.rindex( '/' ) ]
                    date= withoutLastStr[ withoutLastStr.rindex( '/' ) + 1 : len( withoutLastStr ) ]
                    log.writerow({'date':str(date), 'time':'one day', 'new_rows':str(new_rows), 'old_rows':str(old_rows)})   
            
            
def main():
    
    for year in range(2012,2021):

        # case 1: orderflow五分钟线
        f1 = "/dat/all_till20201230/Equity/CN/marketdata/orderflow/5min/data/" + str(year)
        f1_ref= "/dat/all_ref/Equity/CN/marketdata/orderflow/5min/data/" + str(year)
        title1 = '5min'
        
        # case 2: orderflow日线
        f2 = "/dat/all_till20201230/Equity/CN/marketdata/orderflow/daily/data/" + str(year)
        f2_ref= "/dat/all_ref/Equity/CN/marketdata/orderflow/daily/data/" + str(year)
        title2 = 'daily'
        
        # 开个log并运行
        case = 2
        if case == 1:
            files = f1
            files_ref = f1_ref
            title = title1
        if case == 2:
            files = f2
            files_ref = f2_ref
            title = title2
            
        headers = ['date', 'time', 'new_rows', 'old_rows']
        OneYear_log = open("/home/yuqin/work/2020Compare/0114/%s/%s%sCheckRows.csv" %(key, year,title), "w")
        log = csv.DictWriter(OneYear_log, headers)
        log.writeheader()
        print(str(year) + ' log opened')
        result = CheckSize(files, files_ref, case, year, title)
        result.checkrows()
        print(str(year) + ' log done')
    
    
    
if __name__ == '__main__':
    
    main()

### 单一文件统计

In [ ]:
import pandas as pd
import numpy as np
import os
import csv

class CheckSize():
    
    def __init__(self, files, case, year, title, key):
        self.files = files
        self.case = case
        self.year = year
        self.title = title
        self.key = key
        
    def checkrows(self):  #每次调取这个函数是一年的
        
        folder_path = self.files
        DatePath = []  #筛选过的完整路径集合

        datelist = os.listdir(folder_path)   #仅遍历获取该年份下所有的日期文件夹名字的集合
        # orderflow五分钟线
        if (self.case == 1) or (self.case == 3) or (self.case == 5):
            for date_i in datelist:  #筛出两边都有的时间文件
                DatePath.append(folder_path + '/' + date_i) 
                
        # orderflow日线
        if (self.case == 2) or (self.case == 4) or (self.case == 6):
            for date_i in datelist: 
                DatePath.append(folder_path + '/' + date_i + '/' + date_i + '.txt') 
    
        headers = ['date', 'time', 'rows']
        OneYear_log = open("/home/yuqin/work/2020Compare/0115/%s/%s%sCheckRows.csv" %(self.key,self.year,self.title), "a")
        log = csv.DictWriter(OneYear_log, headers)
        for n in range(len(DatePath)):
            
            if (self.case == 1) or (self.case == 3) or (self.case == 5):
                TimePath = []
                timelist = os.listdir(DatePath[n]) 
                for time_i in timelist:
                    TimePath.append(DatePath[n] + '/' + time_i) 
    
                for m in range(len(TimePath)):
                    getpath = TimePath[m]
                    d = getpath
                    time = d[ d.rindex( '/' ) + 1 : len( d ) ] 
                    withoutLastStr = d[ 0 : d.rindex( '/' ) ]
                    date= withoutLastStr[ withoutLastStr.rindex( '/' ) + 1 : len( withoutLastStr ) ]
                    if os.path.exists(getpath) == True:
                        df = pd.read_csv(getpath, sep = ',')
                        df.set_index('SecuCode', inplace=True)
                        rows = len(df.index)
                        log.writerow({'date':str(date), 'time':str(time), 'rows':str(rows)})
                    else:
                        print(date)
                    
            if (self.case == 2) or (self.case == 4) or (self.case == 6):
                getpath = DatePath[n]
                d = getpath
                time = d[ d.rindex( '/' ) + 1 : len( d ) ] 
                withoutLastStr = d[ 0 : d.rindex( '/' ) ]
                date= withoutLastStr[ withoutLastStr.rindex( '/' ) + 1 : len( withoutLastStr ) ]
                if os.path.exists(getpath) == True:
                    df = pd.read_csv(getpath, sep = ',')
                    df.set_index('SecuCode', inplace=True)
                    rows = len(df.index)                    
                    log.writerow({'date':str(date), 'time':'one day', 'rows':str(rows)})  
                else:
                    print(date)
            
            
def main():
    
    for year in range(2012,2022):

        # case 1: orderflow五分钟线
        f1 = "/dat/all_till20201230/Equity/CN/marketdata/orderflow/5min/data/" + str(year)
        title1 = '5min'
        
        # case 2: orderflow日线
        f2 = "/dat/all_till20201230/Equity/CN/marketdata/orderflow/daily/data/" + str(year)
        title2 = 'daily'
        
        # case 3: level2summary五分钟线
        f3 = "/dat/all_till20201230/Equity/CN/marketdata/level2summary/5min/data/" + str(year)
        title3 = '5min'
        
        # case 4: level2summary日线
        f4 = "/dat/all_till20201230/Equity/CN/marketdata/level2summary/daily/data/" + str(year)
        title4 = 'daily'
        
        # case 5: notmkt五分钟线
        f5 = "/dat/all_till20201230/Equity/CN/5min/data/" + str(year)
        title5 = '5min'
        
        # case 6: notmkt日线
        f6 = "/dat/all_till20201230/Equity/CN/daily/data/" + str(year)
        title6 = 'daily'
        
        # 开个log并运行
        case = 1
        if case == 1:
            files = f1
            title = title1
            key = 'orderflow'
        if case == 2:
            files = f2
            title = title2
            key = 'orderflow'
        if case == 3:
            files = f3
            title = title3
            key = 'level2summary'
        if case == 4:
            files = f4
            title = title4
            key = 'level2summary'
        if case == 5:
            files = f5
            title = title5
            key = 'notmkt'
        if case == 6:
            files = f6
            title = title6
            key = 'notmkt'
            
            
        headers = ['date', 'time', 'rows']
        OneYear_log = open("/home/yuqin/work/2020Compare/0115/%s/%s%sCheckRows.csv" %(key,year,title), "w")
        log = csv.DictWriter(OneYear_log, headers)
        log.writeheader()
        OneYear_log.close()
        print(str(year) + ' log opened')
        result = CheckSize(files, case, year, title, key)
        result.checkrows()
        print(str(year) + ' log done')
    
    
    
if __name__ == '__main__':
    
    main()